# Extract Features from MSR-VTT dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/My Drive/RP/Video-Captioning

/content/drive/My Drive/RP/Video-Captioning


In [1]:
!pip install youtube-dl
from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/yunooooo/gcct/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...", ty='twg')
os.system('pip install git+git://github.com/AWConant/jikanpy.git')
os.system('add-apt-repository -y ppa:jonathonf/ffmpeg-4')
os.system('apt-get update')
os.system('apt install mediainfo')
os.system('apt-get install ffmpeg')
clear_output()
print('Installation finished.')

SyntaxError: invalid syntax (<ipython-input-1-ecbaec99cbe3>, line 5)

In [6]:
#!ffmpeg -i /content/drive/My\ Drive/RP/Video-Captioning/DATA/videos/TrainValVideo/video0.mp4 -vf fps=1 /content/drive/My\ Drive/RP/Video-Captioning/DATA/img/output%06d.png
# !ffmpeg -i /content/drive/My\ Drive/RP/Video-Captioning/DATA/videos/TrainValVideo/video0.mp4 -vf fps=.5 -vf scale=320:-1 /content/drive/My\ Drive/RP/Video-Captioning/DATA/img/screenshot_%04d.png

!ffmpeg -i /content/drive/My\ Drive/RP/Video-Captioning/DATA/videos/TrainValVideo/video0.mp4  -r 4 /content/drive/My\ Drive/RP/Video-Captioning/DATA/demo/name_%04d.png

ffmpeg version 4.3-2~18.04.york0 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version='2~18.04.york0' --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab

In [5]:
pwd

'/content/drive/My Drive/RP/Video-Captioning'

In [4]:
video_dir = './DATA/videos/TrainValVideo'
output_dir = './DATA/frames'
features_out = './DATA/features'
jobs = 16

In [5]:
import os

print(os.path.exists(video_dir))

print(os.path.exists(output_dir))

print(os.path.exists(features_out))

True
True
True


### Extract frames from videos

In [11]:
import csv
import glob
import os
import os.path
import subprocess
from subprocess import call
from joblib import Parallel,delayed

data_file = []

def core_func(video_path):
  print("in core function")
  global data_file
  video_parts = get_video_parts(video_path)
  filename_no_ext, filename = video_parts

    # Only extract if we haven't done it yet. Otherwise, just get
    # the info.
  if not check_already_extracted(video_parts):
    print("in if conditions")
    # Now extract it.
    src = os.path.join(video_dir,filename)
    print("SRC:", src)
    dest = os.path.join(output_dir,filename_no_ext + '-%04d.jpg')
    # print('in', src, dest)
    call(["ffmpeg", "-i", src,"-r", "4", dest])
    # command = ['ffmpeg',
    #            '-i', '"%s"' % src,
    #            '-r', '4'
    #            '"%s"' % dest]
    # command = ' '.join(command)
    # print("FFMPEG", command)
    # output = subprocess.check_output(command, shell=True)
    # Now get how many frames it is.
  nb_frames = get_nb_frames_for_video(video_parts)
  print('written: ',nb_frames)
  data_file.append([filename_no_ext, nb_frames])



def get_nb_frames_for_video(video_parts):
  print("in get nb frames for video function")
  """Given video parts of an (assumed) already extracted video, return
    the number of frames that were extracted."""
  filename_no_ext, _ = video_parts
  generated_files = glob.glob(os.path.join(output_dir, filename_no_ext + '*.jpg'))
  print("generated_files: ", generated_files)
  return len(generated_files)

def get_video_parts(video_path):
  print("in video parts")
  """Given a full path to a video, return its parts."""
  parts = video_path.split(os.path.sep)
#   print(parts)
  filename = parts[4]
  filename_no_ext = filename.split('.')[0]
  return filename_no_ext, filename

def check_already_extracted(video_parts):
  print("in already extraction fun")
  """Check to see if we created the -0001 frame of this file."""
  filename_no_ext, _ = video_parts
  return bool(os.path.exists(os.path.join(output_dir,
                               filename_no_ext + '-0001.jpg')))





In [13]:
jobs=2
print("at vfiles")
vfiles = glob.glob(os.path.join(video_dir, '*.mp4'))
print("vfiles:", vfiles)
print("processing parallel")
results = Parallel(n_jobs=jobs)(delayed(core_func)(video_path) for video_path in vfiles)               
print("results", results)
with open('data_file.csv', 'w') as fout:
    writer = csv.writer(fout)
    writer.writerows(data_file)

print("Extracted and wrote %d video files." % (len(data_file)))

at vfiles
('vfiles:', ['./DATA/videos/TrainValVideo/video0.mp4', './DATA/videos/TrainValVideo/video1.mp4', './DATA/videos/TrainValVideo/video10.mp4', './DATA/videos/TrainValVideo/video100.mp4', './DATA/videos/TrainValVideo/video1000.mp4', './DATA/videos/TrainValVideo/video1001.mp4', './DATA/videos/TrainValVideo/video1002.mp4', './DATA/videos/TrainValVideo/video1003.mp4', './DATA/videos/TrainValVideo/video1004.mp4', './DATA/videos/TrainValVideo/video1005.mp4', './DATA/videos/TrainValVideo/video1006.mp4', './DATA/videos/TrainValVideo/video1007.mp4', './DATA/videos/TrainValVideo/video1008.mp4', './DATA/videos/TrainValVideo/video1009.mp4', './DATA/videos/TrainValVideo/video101.mp4', './DATA/videos/TrainValVideo/video1010.mp4', './DATA/videos/TrainValVideo/video1011.mp4', './DATA/videos/TrainValVideo/video1012.mp4', './DATA/videos/TrainValVideo/video1013.mp4', './DATA/videos/TrainValVideo/video1014.mp4', './DATA/videos/TrainValVideo/video1015.mp4', './DATA/videos/TrainValVideo/video1016.mp4

JoblibOSError: JoblibOSError
___________________________________________________________________________
...........................................................................
/Users/vaibhavpatel/opt/anaconda3/envs/opencv/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = '/Users/vaibhavpatel/opt/anaconda3/envs/opencv/lib/python2.7/site-packages/ipykernel_launcher.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = ''
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

...........................

In [22]:

# data_file above did not work in parallel processing need to write a code to count them manually:
from tqdm import tqdm
vfiles = glob.glob(os.path.join(video_dir, '*.mp4'))
pbar = tqdm(total=len(vfiles))

data_file = []
for video_path in vfiles:
    video_parts = get_video_parts(video_path)
    filename_no_ext, filename = video_parts
    generated_files = glob.glob(os.path.join(output_dir, filename_no_ext + '*.jpg'))
    data_file.append([filename_no_ext, len(generated_files)])
    pbar.update(1)
pbar.close()
with open('data_file.csv', 'w') as fout:
    writer = csv.writer(fout)
    writer.writerows(data_file)

print("Extracted and wrote %d video files." % (len(data_file)))
    

  0%|          | 0/7010 [00:00<?, ?it/s]

in video parts


  0%|          | 3/7010 [00:04<6:15:20,  3.21s/it]

in video parts
in video parts
in video parts


  0%|          | 5/7010 [00:04<4:25:46,  2.28s/it]

...

in video parts
in video parts
in video parts


100%|█████████▉| 7007/7010 [10:19<00:00, 12.01it/s]

in video parts
in video parts


100%|██████████| 7010/7010 [10:20<00:00, 11.30it/s]

in video parts
Extracted and wrote 7010 video files.


In [13]:
!ls DATA/videos

Readme	       video2274.mp4  video3550.mp4  video4826.mp4  video6102.mp4
TrainValVideo  video2275.mp4  video3551.mp4  video4827.mp4  video6103.mp4
video1000.mp4  video2276.mp4  video3552.mp4  video4828.mp4  video6104.mp4
video1001.mp4  video2277.mp4  video3553.mp4  video4829.mp4  video6105.mp4
video1002.mp4  video2278.mp4  video3554.mp4  video4830.mp4  video6106.mp4
video1003.mp4  video2279.mp4  video3555.mp4  video4831.mp4  video6107.mp4
video1004.mp4  video2280.mp4  video3556.mp4  video4832.mp4  video6108.mp4
video1005.mp4  video2281.mp4  video3557.mp4  video4833.mp4  video6109.mp4
video1006.mp4  video2282.mp4  video3558.mp4  video4834.mp4  video6110.mp4
video1007.mp4  video2283.mp4  video3559.mp4  video4835.mp4  video6111.mp4
video1008.mp4  video2284.mp4  video3560.mp4  video4836.mp4  video6112.mp4
video1009.mp4  video2285.mp4  video3561.mp4  video4837.mp4  video6113.mp4
video1010.mp4  video2286.mp4  video3562.mp4  video4838.mp4  video6114.mp4
video1011.mp4  video2287.mp4  video3563

In [ ]:
pwd

'/content/drive/.shortcut-targets-by-id/1XoSrXDjkJG8z0QGSn9PH0IzCGzQY2Mm4/Video-Captioning'

In [8]:
!ls DATA/frames

Readme


### Extract CNN Features

In [14]:
jobs = 1

In [1]:
"""
Class for managing our data.
"""
import csv
import numpy as np
import random
import glob
import os.path
import sys
import operator
import threading
from keras.utils import np_utils

class threadsafe_iterator:
    def __init__(self, iterator):
        self.iterator = iterator
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        with self.lock:
            return next(self.iterator)

def threadsafe_generator(func):
    """Decorator"""
    def gen(*a, **kw):
        return threadsafe_iterator(func(*a, **kw))
    return gen

class DataSet():

    def __init__(self, seq_length=40, class_limit=None, image_shape=(224, 224, 3)):
        """Constructor.
        seq_length = (int) the number of frames to consider
        class_limit = (int) number of classes to limit the data to.
            None = no limit.
        """
        self.seq_length = seq_length
        self.class_limit = class_limit
        self.sequence_path = os.path.join('data', 'sequences')
        self.max_frames = 1000  # max number of frames a video can have for us to use it

        # Get the data.
        self.data = self.get_data()

        # Now do some minor data cleaning.
        self.data = self.clean_data()

        self.image_shape = image_shape

    # @staticmethod
    def get_data(self):
        """Load our data from file."""
        with open(os.path.join('data_file.csv'), 'r') as fin:
            reader = csv.reader(fin)
            data = list(reader)
            print(len(data))

        return data

    def clean_data(self):
        """Limit samples to greater than the sequence length and fewer
        than N frames. Also limit it to classes we want to use."""
        data_clean = []
        for item in self.data:
            if int(item[1]) >= self.seq_length and int(item[1]) <= self.max_frames:
                data_clean.append(item)

        return data_clean


    def build_image_sequence(self, frames):
        """Given a set of frames (filenames), build our sequence."""
        return [process_image(x, self.image_shape) for x in frames]

    def get_extracted_sequence(self, data_type, sample):
        """Get the saved extracted features."""
        print("in sample:", sample)
        filename = sample[2]
        path = os.path.join(self.sequence_path, filename + '-' + str(self.seq_length) + \
            '-' + data_type + '.npy')
        if os.path.isfile(path):
            return np.load(path)
        else:
            return None

    @staticmethod
    def get_frames_for_sample(sample):
        """Given a sample row from the data file, get all the corresponding frame
        filenames."""
        filename = sample[0]
        images = sorted(glob.glob(os.path.join(output_dir, filename + '*jpg')))
        return images

    @staticmethod
    def get_filename_from_image(filename):
        parts = filename.split(os.path.sep)
        return parts[-1].replace('.jpg', '')

    @staticmethod
    def rescale_list(input_list, size):
        """Given a list and a size, return a rescaled/samples list. For example,
        if we want a list of size 5 and we have a list of size 25, return a new
        list of size five which is every 5th element of the origina list."""
        assert len(input_list) >= size

        # Get the number to skip between iterations.
        skip = len(input_list) // size

        # Build our new output.
        output = [input_list[i] for i in range(0, len(input_list), skip)]

        # Cut off the last one if needed.
        return output[:size]

Using TensorFlow backend.


In [2]:
from keras.preprocessing.image import img_to_array, load_img
import numpy as np

def process_image(image, target_shape):
    """Given an image, process it and return the array."""
    # Load the image.
    h, w, _ = target_shape
    image = load_img(image, target_size=(h, w))

    # Turn it into numpy, normalize and return.
    img_arr = img_to_array(image)
    x = (img_arr / 255.).astype(np.float32)

    return x

In [3]:
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model
from keras.layers import Input
import numpy as np

class Extractor():
    def __init__(self, weights=None):
        """Either load pretrained from imagenet, or load our saved
        weights from our own training."""

        self.weights = weights  # so we can check elsewhere which model

        if weights is None:
            # Get model with pretrained weights.
            base_model = InceptionV3(
                weights='imagenet',
                include_top=True
            )

            # We'll extract features at the final pool layer.
            self.model = Model(
                inputs=base_model.input,
                outputs=base_model.get_layer('avg_pool').output
            )

        else:
            # Load the model first.
            self.model = load_model(weights)

            # Then remove the top so we get features not predictions.
            # From: https://github.com/fchollet/keras/issues/2371
            self.model.layers.pop()
            self.model.layers.pop()  # two pops to get to pool layer
            self.model.outputs = [self.model.layers[-1].output]
            self.model.output_layers = [self.model.layers[-1]]
            self.model.layers[-1].outbound_nodes = []

    def extract(self, image_path):
        img = image.load_img(image_path, target_size=(299, 299))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        # Get the prediction.
        features = self.model.predict(x)

        if self.weights is None:
            # For imagenet/default network:
            features = features[0]
        else:
            # For loaded network:
            features = features[0]

        return features

In [4]:
seq_length = 30
class_limit = None  # Number of classes to extract. Can be 1-101 or None for all.

# Get the dataset.
data = DataSet(seq_length=seq_length, class_limit=class_limit)
csv_feature = data.get_data()
print("size",len(csv_feature))
frames = data.get_frames_for_sample(video)
print(frames)

14020


IndexError: list index out of range

In [11]:
import numpy as np
import os.path
from tqdm import tqdm

# Set defaults.
seq_length = 30
class_limit = None  # Number of classes to extract. Can be 1-101 or None for all.

# Get the dataset.
data = DataSet(seq_length=seq_length, class_limit=class_limit)

# get the model.
model = Extractor()

print(len(csv_feature))
# Loop through data.
pbar = tqdm(total=len(csv_feature))
for video in csv_feature:

    # Get the path to the sequence for this video.
    path = os.path.join(features_out, video[0] + '-' + str(seq_length) + \
        '-features')  # numpy will auto-append .npy

    # Check if we already have it.
    if os.path.isfile(path):
        pbar.update(1)
        continue

    # Get the frames for this video.
    frames = data.get_frames_for_sample(video)
    
    # Now downsample to just the ones we need.
    frames = data.rescale_list(frames, seq_length)
    # print("Frames", frames)
    # Now loop through and extract features to build the sequence.
    sequence = []
    for frame in frames:
        features = model.extract(frame)
        sequence.append(features)
    
    
    # Save the sequence.
    np.save(path, sequence)
    
    pbar.update(1)

pbar.close()

7010





  0%|          | 0/7010 [00:00<?, ?it/s]

7010


AssertionError: ignored

In [ ]:
!sudo pip3 install h5py

In [ ]:
!sudo apt-get install libhdf5-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libhdf5-dev is already the newest version (1.10.0-patch1+docs-4).
0 upgraded, 0 newly installed, 0 to remove and 50 not upgraded.
